In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/content/drive/MyDrive/qoute_dataset.csv")

In [3]:
df.head()

,quote,Author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe


In [4]:
df.shape

(3038, 2)

In [5]:
quotes = df['quote']
quotes.head()

,quote
0,“The world as we have created it is a process ...
1,"“It is our choices, Harry, that show what we t..."
2,“There are only two ways to live your life. On...
3,"“The person, be it gentleman or lady, who has ..."
4,"“Imperfection is beauty, madness is genius and..."


In [6]:
quotes[0]

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

In [7]:
quotes = quotes.str.lower()

In [8]:
quotes[0]

'“the world as we have created it is a process of our thinking. it cannot be changed without changing our thinking.”'

In [9]:
import string
translator = str.maketrans('', '', string.punctuation)
quotes = quotes.apply(lambda x: x.translate(translator))

In [10]:
quotes[0]

'“the world as we have created it is a process of our thinking it cannot be changed without changing our thinking”'

## tokenisation

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [12]:
vocab_size = 10000
tokinizer = Tokenizer(num_words=vocab_size)
tokinizer.fit_on_texts(quotes)

In [13]:
word_index = tokinizer.word_index
print(len(word_index))

8978


In [14]:
list(word_index.items())[:20]

[('the', 1),
 ('you', 2),
 ('to', 3),
 ('and', 4),
 ('a', 5),
 ('i', 6),
 ('is', 7),
 ('of', 8),
 ('that', 9),
 ('it', 10),
 ('in', 11),
 ('be', 12),
 ('not', 13),
 ('are', 14),
 ('your', 15),
 ('have', 16),
 ('for', 17),
 ('but', 18),
 ('we', 19),
 ('if', 20)]

## Vectorisation

In [15]:
sequence = tokinizer.texts_to_sequences(quotes)

In [16]:
print(word_index.get('“the'))

713


In [17]:
quotes[0]

'“the world as we have created it is a process of our thinking it cannot be changed without changing our thinking”'

In [18]:
sequence[0]

[713,
 62,
 29,
 19,
 16,
 946,
 10,
 7,
 5,
 1156,
 8,
 70,
 293,
 10,
 145,
 12,
 809,
 104,
 752,
 70,
 2461]

In [19]:
X = []
y = []

for seq in sequence:
  for i in range(1,len(seq)):
    input_seq = seq[:i]
    output_seq = seq[i]
    X.append(input_seq)
    y.append(output_seq)
# ye code Next Word Prediction (Language Modeling) ke liye hi likha jata hai.

In [20]:
X

[[713],
 [713, 62],
 [713, 62, 29],
 [713, 62, 29, 19],
 [713, 62, 29, 19, 16],
 [713, 62, 29, 19, 16, 946],
 [713, 62, 29, 19, 16, 946, 10],
 [713, 62, 29, 19, 16, 946, 10, 7],
 [713, 62, 29, 19, 16, 946, 10, 7, 5],
 [713, 62, 29, 19, 16, 946, 10, 7, 5, 1156],
 [713, 62, 29, 19, 16, 946, 10, 7, 5, 1156, 8],
 [713, 62, 29, 19, 16, 946, 10, 7, 5, 1156, 8, 70],
 [713, 62, 29, 19, 16, 946, 10, 7, 5, 1156, 8, 70, 293],
 [713, 62, 29, 19, 16, 946, 10, 7, 5, 1156, 8, 70, 293, 10],
 [713, 62, 29, 19, 16, 946, 10, 7, 5, 1156, 8, 70, 293, 10, 145],
 [713, 62, 29, 19, 16, 946, 10, 7, 5, 1156, 8, 70, 293, 10, 145, 12],
 [713, 62, 29, 19, 16, 946, 10, 7, 5, 1156, 8, 70, 293, 10, 145, 12, 809],
 [713, 62, 29, 19, 16, 946, 10, 7, 5, 1156, 8, 70, 293, 10, 145, 12, 809, 104],
 [713,
  62,
  29,
  19,
  16,
  946,
  10,
  7,
  5,
  1156,
  8,
  70,
  293,
  10,
  145,
  12,
  809,
  104,
  752],
 [713,
  62,
  29,
  19,
  16,
  946,
  10,
  7,
  5,
  1156,
  8,
  70,
  293,
  10,
  145,
  12,
  809,
  

In [21]:
len(X)

85271

In [22]:
len(y)

85271

len(X) == len(y) -->Har input ke liye ek output hai

In [23]:
max_len = max(len(x) for x in X)
print(max_len)

745


->hmare dataset me sabse lamba input sequence 745 words ka hai.

->Kisi ek quote me (ya combined sequence me) 746 words the Isliye max input length = 745 (kyunki last word output ban gaya)

->Neural Networks fixed-size input lete hain.
->hmare sequences different length ke h,Isliye unhe same length ka banana zaroori h

# Padding

In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_padded = pad_sequences(X, maxlen=max_len, padding='pre')

In [25]:
y = np.array(y)

In [26]:
X_padded.shape

(85271, 745)

85271 → Total training samples

745 → Har sample ki length (max_len)

In [27]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(y, num_classes=vocab_size)

In [28]:
y.shape

(85271,)

In [29]:
y_one_hot.shape

(85271, 10000)

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,LSTM, Dense

In [31]:
embedding_dim = 50
rnn_units = 128


In [32]:
rnn_model = Sequential()

rnn_model.add(
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)
)
rnn_model.add(SimpleRNN(units=rnn_units))
rnn_model.add(Dense(units=vocab_size, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [33]:
rnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [34]:
rnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [35]:
lstm_model = Sequential()
lstm_model.add(
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)
)
lstm_model.add(LSTM(units=rnn_units))
lstm_model.add(Dense(units=vocab_size, activation='softmax'))

In [36]:
lstm_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [37]:
lstm_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [38]:
epochs=100
batch_size=128
history_lstm=lstm_model.fit(
    X_padded,
    y_one_hot,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1
)

Epoch 1/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 38s 53ms/step - accuracy: 0.0336 - loss: 7.1044 - val_accuracy: 0.0444 - val_loss: 6.6918
Epoch 2/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - accuracy: 0.0520 - loss: 6.3764 - val_accuracy: 0.0610 - val_loss: 6.6256
Epoch 3/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.0688 - loss: 6.1517 - val_accuracy: 0.0786 - val_loss: 6.5129
Epoch 4/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 52ms/step - accuracy: 0.0877 - loss: 5.9021 - val_accuracy: 0.0945 - val_loss: 6.4604
Epoch 5/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 52ms/step - accuracy: 0.1027 - loss: 5.7250 - val_accuracy: 0.0978 - val_loss: 6.4610
Epoch 6/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.1119 - loss: 5.5594 - val_accuracy: 0.1025 - val_loss: 6.4563
Epoch 7/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 52ms/step - accuracy: 0.1239 - loss: 5.3970 - val_accuracy: 0.1059 - val_loss: 6.4721
Epoch 8/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 52ms/step - accuracy: 0.1309 - loss: 5

In [39]:
lstm_model.save("/content/drive/MyDrive/lstm_model.h5")


In [40]:
index_to_word = {}
for word, index in word_index.items():
  index_to_word[index] = word


In [41]:
index_to_word

{1: 'the',
 2: 'you',
 3: 'to',
 4: 'and',
 5: 'a',
 6: 'i',
 7: 'is',
 8: 'of',
 9: 'that',
 10: 'it',
 11: 'in',
 12: 'be',
 13: 'not',
 14: 'are',
 15: 'your',
 16: 'have',
 17: 'for',
 18: 'but',
 19: 'we',
 20: 'if',
 21: 'what',
 22: 'with',
 23: 'all',
 24: 'love',
 25: 'can',
 26: 'my',
 27: 'when',
 28: 'will',
 29: 'as',
 30: 'who',
 31: 'do',
 32: 'or',
 33: 'me',
 34: 'he',
 35: 'they',
 36: 'life',
 37: 'one',
 38: 'was',
 39: 'like',
 40: 'there',
 41: 'people',
 42: 'on',
 43: 'its',
 44: 'at',
 45: 'so',
 46: 'never',
 47: 'no',
 48: 'them',
 49: 'dont',
 50: 'know',
 51: 'just',
 52: 'more',
 53: 'only',
 54: 'than',
 55: 'because',
 56: 'this',
 57: 'want',
 58: 'up',
 59: 'how',
 60: 'his',
 61: 'things',
 62: 'world',
 63: 'by',
 64: 'think',
 65: 'make',
 66: 'about',
 67: 'time',
 68: 'from',
 69: 'always',
 70: 'our',
 71: 'an',
 72: 'out',
 73: 'us',
 74: 'good',
 75: 'said',
 76: 'she',
 77: 'her',
 78: 'way',
 79: 'go',
 80: 'am',
 81: 'live',
 82: 'has',
 83:

In [42]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [43]:
def predictor(model,tokenizer,text,max_len):
  text = text.lower()

  seq = tokenizer.texts_to_sequences([text])[0]
  seq = pad_sequences([seq], maxlen=max_len, padding='pre')

  pred = model.predict(seq,verbose = 0)
  pred_index = np.argmax(pred)
  return index_to_word[pred_index]

In [44]:
seed_text = "what are you"
next_word = predictor(lstm_model,tokinizer,seed_text,max_len)
print(next_word)

worrying


In [51]:
#to predict full sentence
def generate_text(model,tokenizer,seed_text,max_len,n_words):
  for _ in range(n_words):
    next_word = predictor(model,tokenizer,seed_text,max_len)
    if next_word == "":
      break
    seed_text += " " + next_word
  return seed_text

In [52]:
seed = "are you a "
generated_text_output = generate_text(lstm_model,tokinizer,seed,max_len,10)
print(generated_text_output)

are you a  boy who was good enough to make you a dictionary


In [53]:
seed = "who are u "
generate_text = generate_text(lstm_model,tokinizer,seed,max_len,10)
print(generate_text)

who are u  the first shadowhunter ive ever met “that’s too bad they


In [54]:
import pickle
with open("/content/drive/MyDrive/tokenizer.pkl", "wb") as f:
  pickle.dump(tokinizer, f)